# Metaclasses

Superclass of `metaclass` is `type`, ONLY metaclasses are BOTH instances AND subclasses of type, all other classes are just instances

## `__init__` vs `__new__`

`__new__()` is intended mainly to allow subclasses of immutable types (like int, str, or tuple) to customize instance creation. It is also commonly overridden in custom metaclasses in order to customize class creation.

Because `__new__()` and `__init__()` work together in constructing objects (`__new__()` to create it, and `__init__()` to customize it), no non-None value may be returned by __init__(); doing so will cause a TypeError to be raised at runtime.

Function signatures:  
Metaclass `__new__`:  
`__new__(cls, classname, bases, namespace)`  

## Metaclass Methods
- `Metaclass.__prepare__` just returns the namespace object (a dictionary-like object as explained before).
- `Metaclass.__new__` returns the Class object.
- `Metaclass.__call__` returns whatever Metaclass.__new__ returned (and if it returned an instance of Metaclass it will also call Metaclass.__init__ on it).

## Types
Classes allow you to make instances of the class (i.e. new objects), but classes are objects

What is `type` of class?
 Class definition is a syntactic way of creating a new type

Metclasses help customise what happens in the `prepare` method or the `__init__`/`__new__` methods.

Metaclasses are inheritable

An object’s type determines the operations that the object supports (e.g., “does it have a length?”) and also defines the possible values for objects of that type

In [1]:
class Parent:
    pass

class Child(Parent):
    pass

type(Child)

type

In [ ]:
type(object), type(type) # object is an instance of type, type is a subclass of object, type is an instance of itself!! #wtf

In [ ]:
# classes are objects, therefore each class must be an instance of some other class 🤯
type.__class__ is type(type)

`__new__` must always return an object instance by calling `type.__new__`, else the object is not created:

In [2]:
class MetaClassOne(type):
    def __new__(cls, *args):
        print(cls)
        print(args)

        # this line ensures the cls __init__ is called
        return type.__new__(cls, *args)


# this is here to show the execution order
class ExampleClass(metaclass=MetaClassOne):
    int1 = 123

    def test():
        print('test')

# this output is at time of CLASS creation NOT instance creation!!

<class '__main__.MetaClassOne'>
('ExampleClass', (), {'__module__': '__main__', '__qualname__': 'ExampleClass', 'int1': 123, 'test': <function ExampleClass.test at 0x7fbc384eb3a0>})


# Class Creation

When a class definition is executed, the following steps occur:
- MRO entries are resolved;
- the appropriate metaclass is determined;
- the class namespace is prepared;
- the class body is executed;
- the class object is created.


In [3]:
# signature is type(classname, bases, namespace)
B = type('B', (), {}) # no difference between type and class
print(f'{B=}')

B=<class '__main__.B'>


The following methods are equivalent for creating a class

In [4]:
# either
class A:
    pass

# OR 
def make_A():
    name = 'A'
    bases = ()

    a = 1
    b = 'hello'


    namespace = type.__prepare__(name, bases)
    body = (
'''
a = 1
b = 'hello'

def f(self,):
    return 117
'''
    )
    exec(body, globals(), namespace)

    A = type(name, bases, namespace)
    return A

A = make_A()
a = A()
a.f()

117

More details:

In [ ]:
field_names = tuple()
def __init__(self, *args, **kwargs):
    attrs = dict(zip(self.__slots__, args))
    attrs.update(kwargs)
    for name, value in attrs.items():
        setattr(self, name, value)
def __iter__(self):
    for name in self.__slots__:
        yield getattr(self, name)
def __repr__(self):
    values = ', '.join('{}={!r}'.format(*i) for i in zip(self.__slots__, self))
    return '{}({})'.format(self.__class__.__name__, values)

cls_attrs = dict(
    __slots__ = field_names,
    __init__ = __init__,
    __iter__ = __iter__,
    __repr__ = __repr__)

# how to make a class
cls_name='NEW_CLASS'
type(cls_name, (object, ), cls_attrs)

## Namespaces / PEP 3115

When class created, the various functions are defined and stored in some diciontary. Can edit this dictionary by overriding default `__prepare__` method

Namespaces in Python. A namespace is a collection of currently defined symbolic names along with information about the object that each name references. You can think of a namespace as a dictionary in which the keys are the object names and the values are the objects themselves

In [ ]:
# prepare happens before new, which happens before init
from typing import Any, Mapping

class Meta(type):
    @classmethod
    def __prepare__(
        metacls,
        __name: str,
        __bases: tuple[type, ...], 
        **kwds: Any) -> Mapping[str, object]:

        # return super().__prepare__(__name, __bases, **kwds) # default
        return collections.OrderedDict()

    def __init__(cls, __name, __bases, __attr_dict):
        super().__init__(__name, __bases, __attr_dict)
        cls._field_names = []
        
        for key, attr in __attr_dict.items():
            if isinstance(attr, Validated):
                type_name = type(attr).__name__

Example usages of `__prepare__`:   
[Pandas Library: to register holidays](https://github.com/pandas-dev/pandas/blob/2be9661853f4e425e00e3a32d265fe889b242f44/pandas/tseries/holiday.py)  
[cuDF overriding methods](https://github.com/rapidsai/cudf/blob/6ca2ceb8e200d55f1f681a4ca086614a28d67ad1/python/cudf/cudf/core/index.py)  
[cuML for fixing init](https://github.com/rapidsai/cuml/blob/50716cf98c4103aa8dbbcc4ea64897ccb7a70722/python/cuml/internals/base_helpers.py)

# Example Usages

## Enforcing Function Implementation
From [James Powell: Advanced Metaphors in Coding with Python](https://www.youtube.com/watch?v=R2ipPgrWypI&t=446s)

In [7]:
# OPTION I: Testing for method existence using assert
class Base:
    def bar(self, ):
        return 'bar'

assert hasattr(Base, 'bar')
# OR using a unit test that
def test_Base():
    # this fulfills all implicit assumptions about my code if it passes
    b = Base()
    b.bar

# OPTION II: With a Metaclass
# metaclasses construct the new classes
# __new__ creates a new CLASS, NOT AN INSTANCE

class BaseMeta(type):
    def __new__(cls, name, bases, body):
        # example
        if 'baz' not in body:
            raise TypeError('baz method must be implemented')
        return super().__new__(cls, name, bases, body)

class Base(metaclass=BaseMeta):
    def baz(self, ):
        pass

    def bar(self,):
        return self.baz()

## MetaBunch

From Python in a Nutshell

In [ ]:
# Example 4-1. The MetaBunch metaclass
import collections
import warnings
class MetaBunch(type):
    """
    Metaclass for new and improved "Bunch": implicitly defines
    __slots__, __init__ and __repr__ from variables bound in
    class scope.
    A class statement for an instance of MetaBunch (i.e., for a
    class whose metaclass is MetaBunch) must define only
    class-scope data attributes (and possibly special methods, but
    NOT __init__ and __repr__). MetaBunch removes the data
    attributes from class scope, snuggles them instead as items in
    a class-scope dict named __dflts__, and puts in the class a
    __slots__ with those attributes' names, an __init__ that takes
    as optional named arguments each of them (using the values in
    __dflts__ as defaults for missing ones), and a __repr__ that
    shows the repr of each attribute that differs from its default
    value (the output of __repr__ can be passed to __eval__ to make
    an equal instance, as per usual convention in the matter, if
    each non-default-valued attribute respects the convention too).
    In v3, the order of data attributes remains the same as in the
    class body; in v2, there is no such guarantee.
    """
    def __prepare__(name, *bases, **kwargs):
        # precious in v3—harmless although useless in v2
        return collections.OrderedDict()

    def __new__(mcl, classname, bases, classdict):
        """ Everything needs to be done in __new__, since
        type.__new__ is where __slots__ are taken into account.
        """
        # define as local functions the __init__ and __repr__ that
        # we'll use in the new class
        def __init__(self, **kw):
            """ Simplistic __init__: first set all attributes to
            default values, then override those explicitly
            passed in kw.
            """
            for k in self.__dflts__:
                setattr(self, k, self.__dflts__[k])
            for k in kw:
                setattr(self, k, kw[k])

        def __repr__(self):
            """ Clever __repr__: show only attributes that differ
            from default values, for compactness.
            """
            rep = ['{}={!r}'.format(k, getattr(self, k))
            for k in self.__dflts__
            if getattr(self, k) != self.__dflts__[k]
            ]
            return '{}({})'.format(classname, ', '.join(rep))

        # build the newdict that we'll use as class-dict for the
        # new class
        newdict = { '__slots__':[],
        '__dflts__':collections.OrderedDict(),
        '__init__':__init__, '__repr__':__repr__, }
        for k in classdict:
            if k.startswith('__') and k.endswith('__'):
                # dunder methods: copy to newdict, or warn
                # about conflicts
                if k in newdict:
                    warnings.warn("Can't set attr {!r} in bunch-class {!r}".format(k, classname))
                else:
                    newdict[k] = classdict[k]
            else:
                # class variables, store name in __slots__, and
                # name and value as an item in __dflts__
                newdict['__slots__'].append(k)
                newdict['__dflts__'][k] = classdict[k]
        # finally delegate the rest of the work to type.__new__
        return super(MetaBunch, mcl).__new__(mcl, classname, bases, newdict)

class Bunch(metaclass=MetaBunch):
    pass

## Registry for Neural Network Layers

In this example, the LayerMetaclass is used to create a custom neural network layer class. The LayerMetaclass adds the custom layer class to a registry of available layers, which can be used to easily create instances of the custom layer in a neural network model. The custom layer class itself defines the forward and backward methods that perform the forward and backward passes for the layer, respectively.

This is just one example of how metaclasses can be used in machine learning, but there are many other potential applications for this powerful feature of Python.

In [ ]:
# Define a metaclass for creating custom neural network layers
class LayerMetaclass(type):
    def __new__(cls, name, bases, attrs):
        # Create a new class for the layer
        layer_class = super(LayerMetaclass, cls).__new__(cls, name, bases, attrs)

        # Add the layer class to a registry of available layers
        LAYER_REGISTRY[name] = layer_class

        return layer_class

# Define a base class for neural network layers
class Layer(metaclass=LayerMetaclass):
    def __init__(self, **kwargs):
        # Initialize the layer with any provided keyword arguments
        self.params = kwargs

    def forward(self, inputs):
        # Perform the forward pass for the layer
        raise NotImplementedError()

    def backward(self, grads):
        # Perform the backward pass for the layer
        raise NotImplementedError()

# Define a custom neural network layer
class MyCustomLayer(Layer):
    def forward(self, inputs):
        # Perform the forward pass for the layer
        return inputs * self.params['scale']

    def backward(self, grads):
        # Perform the backward pass for the layer
        return grads * self.params['scale']

# Create an instance of the custom layer
my_layer = MyCustomLayer(scale=2.0)

# Use the layer in a neural network model
outputs = my_layer.forward(inputs)

## Singleton

In [8]:
# Define a metaclass for creating singleton classes
class SingletonMetaclass(type):
    def __new__(cls, name, bases, attrs):
        # Create a new class for the singleton
        singleton_class = super(SingletonMetaclass, cls).__new__(cls, name, bases, attrs)

        # Create a class variable for storing the singleton instance
        singleton_class.instance = None

        return singleton_class

    def __call__(cls, *args, **kwargs):
        # Return the singleton instance if it exists, otherwise create it
        if cls.instance is None:
            cls.instance = super(SingletonMetaclass, cls).__call__(*args, **kwargs)
        return cls.instance

# Define a base class for singletons
class Singleton(metaclass=SingletonMetaclass):
    pass

# Define a custom singleton class
class MyCustomSingleton(Singleton):
    def __init__(self, value):
        self.value = value

# Create multiple instances of the custom singleton class
s1 = MyCustomSingleton(1)
s2 = MyCustomSingleton(2)
s3 = MyCustomSingleton(3)

# All instances of the singleton class will be the same
assert s1 == s2 == s3


## Logging

Adds logs to EVERY method

In [ ]:

# Define a metaclass for creating classes with logging
class LoggingMetaclass(type):
    def __new__(cls, name, bases, attrs):
        # Create a new class for the object
        logging_class = super(LoggingMetaclass, cls).__new__(cls, name, bases, attrs)

        # Add a method for logging method calls
        def log_method_call(self, method_name, *args, **kwargs):
            print('Calling method {} with arguments {}, {}'.format(method_name, args, kwargs))
            return method(*args, **kwargs)

        # Override each method in the class with a logged version of the method
        for method_name, method in attrs.items():
            if callable(method):
                setattr(logging_class, method_name, log_method_call)

        return logging_class

# Define a base class for objects with logging
class LoggingObject(metaclass=LoggingMetaclass):
    pass

# Define a custom class with logging
class MyCustomObject(LoggingObject):
    def my_method(self, arg1, arg2):
        print('Inside my_method with arguments {}, {}'.format(arg1, arg2))

# Create an instance of the custom class
my_object = MyCustomObject()

# Call the method on the object
my_object.my_method(1, 2)

ChatGPT uses:
Here are ten potential uses for custom metaclasses in Python:

- Creating registry classes for storing and accessing instances of other classes.
- Implementing the Singleton design pattern to ensure that only one instance of a class is created.
- Automatically logging method calls to track the behavior of an object.
- Enforcing constraints on the attributes and methods of a class.
- Providing default implementations for common methods in a class hierarchy.
- Creating DSLs (domain-specific languages) for specific problem domains.
- Implementing the decorator pattern for modifying the behavior of class methods.
- Implementing the factory pattern for creating instances of classes based on specified criteria.
- Creating classes that can be easily serialized and deserialized for storage or communication.
- Creating classes with custom behavior when they are used in arithmetic operations.
- Creating classes that can be easily extended with additional functionality using mixins.
- Implementing the observer pattern for automatically updating dependent objects when the state of an object changes.
- Generating boilerplate code for common class patterns, such as getters and setters for attributes.
- Creating classes that can be easily subclassed and customized by other developers.
- Creating classes that automatically cache the results of expensive computations.
- Implementing the command pattern for creating objects that represent actions that can be executed and undone.
- Creating classes that automatically register themselves with a central registry or manager.
- Implementing the template method pattern for defining a common algorithm that can be customized by subclasses.
- Creating classes that can be easily serialized to and deserialized from a specific data format, such as JSON or XML.
- Creating classes that automatically generate and update documentation for their attributes and methods.